In [57]:
import pandas as pd
import time
import numpy as np
from nba_api.stats.endpoints import playercareerstats
import nba_api.stats.endpoints as nbaapi
from nba_api.stats.endpoints import leaguegamefinder
import psycopg2
import re
import allintwo

conn = allintwo.connect_to_rds('thebigone', 'ajwin', 'CharlesBark!23', 'nba-rds-instance.c9wwc0ukkiu5.us-east-1.rds.amazonaws.com')


Connected to RDS PostgreSQL database


In [ ]:
from psycopg2 import pool

# Create a connection pool
db_pool = pool.SimpleConnectionPool(
    minconn=1,   # Minimum number of connections
    maxconn=10,  # Maximum number of connections
    user='your_username',
    password='your_password',
    host='your_host',
    port='your_port',
    database='your_database'
)

# Get a connection from the pool
connection = db_pool.getconn()

try:
    # Use the connection
    with connection.cursor() as cursor:
        cursor.execute("SELECT * FROM your_table;")
        result = cursor.fetchall()
        print(result)
finally:
    # Return the connection to the pool
    db_pool.putconn(connection)

# Close the pool when done (e.g., at application shutdown)
db_pool.closeall()


In [11]:
    
query = "SELECT * FROM pg_catalog.pg_tables where tableowner = 'ajwin';"  # Example query: fetch 10 rows from the 'games' table

if conn:
    result_df = allintwo.query_database_to_dataframe(conn, query)

result_df

c:\Users\ajwin\Projects\NBA\thebigone\allintwo.py:103: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframe = pd.read_sql(query, conn)


,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,public,masterplayers,ajwin,None,False,False,False,False
1,public,mastergames,ajwin,None,False,False,False,False
2,public,boxscoreadvancedv3_0,ajwin,None,False,False,False,False


In [18]:
    
conn = allintwo.connect_to_rds('thebigone', 'ajwin', 'CharlesBark!23', 'nba-rds-instance.c9wwc0ukkiu5.us-east-1.rds.amazonaws.com')

query = "SELECT * FROM mastergames;"  # Example query: fetch 10 rows from the 'games' table

result_df = allintwo.query_database_to_dataframe(conn, query)

conn.close()

result_df


Connected to RDS PostgreSQL database


c:\Users\ajwin\Projects\NBA\thebigone\allintwo.py:103: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframe = pd.read_sql(query, conn)


,seasonid,teamid,teamabbreviation,teamname,gameid,gamedate,matchup,wl,min,pts,...,ftpct,oreb,dreb,reb,ast,stl,blk,tov,pf,plusminus
0,52024,1612709926,MHU,Memphis Hustle,2052400089,2024-11-23,MHU vs. TEX,W,242,119,...,0.529,19.0,37.0,56,21,9.0,3,17,17,23.8
1,22024,1610612744,GSW,Golden State Warriors,0022400269,2024-11-23,GSW @ SAS,L,241,94,...,0.615,17.0,32.0,49,29,4.0,3,10,21,-10.0
2,52024,1612709927,WIS,Wisconsin Herd,2052400087,2024-11-23,WIS @ WCB,W,239,86,...,0.909,4.0,38.0,42,21,10.0,12,15,19,9.0
3,52024,1612709902,SCW,Santa Cruz Warriors,2052400091,2024-11-23,SCW @ SLC,L,241,102,...,0.364,13.0,28.0,41,23,2.0,3,17,19,-33.4
4,52024,1612709917,GRG,Grand Rapids Gold,2052400088,2024-11-23,GRG @ IWA,L,240,88,...,0.615,7.0,23.0,30,20,13.0,7,15,23,-28.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123767,21983,1610612746,SDC,San Diego Clippers,0028300007,1983-10-28,SDC @ POR,L,240,93,...,0.750,7.0,24.0,31,19,5.0,5,18,31,NaN
123768,21983,1610612764,WAS,Washington Bullets,0028300003,1983-10-28,WAS @ PHL,L,240,114,...,0.759,10.0,26.0,36,32,9.0,3,21,35,NaN
123769,21983,1610612747,LAL,Los Angeles Lakers,0028300002,1983-10-28,LAL @ KCK,W,240,117,...,0.760,9.0,29.0,38,22,6.0,5,17,16,NaN
123770,21983,1610612758,KCK,Kansas City Kings,0028300002,1983-10-28,KCK vs. LAL,L,240,107,...,0.739,21.0,24.0,45,17,5.0,6,22,25,NaN


In [8]:
### CHECK TABLES
conn = allintwo.connect_to_rds('thebigone', 'ajwin', 'CharlesBark!23', 'nba-rds-instance.c9wwc0ukkiu5.us-east-1.rds.amazonaws.com')

cursor = conn.cursor()

cursor.execute("""
        SELECT tablename
        FROM pg_tables
        WHERE schemaname = 'public';
    """)


cursor.fetchall()

Connected to RDS PostgreSQL database


[('masterplayers',), ('mastergames',), ('boxscoreadvancedv3_0',)]

In [16]:
### DROP TABLES
conn = allintwo.connect_to_rds('thebigone', 'ajwin', 'CharlesBark!23', 'nba-rds-instance.c9wwc0ukkiu5.us-east-1.rds.amazonaws.com')
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS boxscoreadvancedv3_0;")
conn.commit()
conn.close()

Connected to RDS PostgreSQL database


In [58]:
#conn = allintwo.connect_to_rds('thebigone', 'ajwin', 'CharlesBark!23', 'nba-rds-instance.c9wwc0ukkiu5.us-east-1.rds.amazonaws.com')

query = "SELECT * FROM boxscoreadvancedv3_0 WHERE personid = 1629652;"  # Example query: fetch 10 rows from the 'games' table

result_df = allintwo.query_database_to_dataframe(conn, query)

result_df

c:\Users\ajwin\Projects\NBA\thebigone\allintwo.py:103: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframe = pd.read_sql(query, conn)


,gameid,teamid,teamcity,teamname,teamtricode,teamslug,personid,firstname,familyname,namei,...,turnoverratio,effectivefieldgoalpercentage,trueshootingpercentage,usagepercentage,estimatedusagepercentage,estimatedpace,pace,paceper40,possessions,pie
0,2021900341,1612709889,Oklahoma City,Blue,OKL,blue,1629652,Luguentz,Dort,L. Dort,...,0.0,0.000,0.000,0.000,0.000,0.00,0.00,0.00,0.0,0.000
1,0022200435,1610612760,Oklahoma City,Thunder,OKC,thunder,1629652,Luguentz,Dort,L. Dort,...,15.4,0.214,0.214,0.153,0.151,107.68,105.27,87.72,51.0,-0.018
2,0022300340,1610612760,Oklahoma City,Thunder,OKC,thunder,1629652,Luguentz,Dort,L. Dort,...,0.0,0.625,0.666,0.175,0.177,95.55,95.88,79.90,52.0,0.068
3,0022000575,1610612760,Oklahoma City,Thunder,OKC,thunder,1629652,Luguentz,Dort,L. Dort,...,7.7,0.667,0.709,0.157,0.160,97.87,96.47,80.40,61.0,0.087
4,0021901292,1610612760,Oklahoma City,Thunder,OKC,thunder,1629652,Luguentz,Dort,L. Dort,...,9.1,0.313,0.429,0.208,0.205,105.45,103.77,86.48,45.0,0.046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,0012300038,1610612760,Oklahoma City,Thunder,OKC,thunder,1629652,Luguentz,Dort,L. Dort,...,0.0,0.000,0.000,0.000,0.000,0.00,0.00,0.00,0.0,0.000
137,0022100354,1610612760,Oklahoma City,Thunder,OKC,thunder,1629652,Luguentz,Dort,L. Dort,...,4.5,0.605,0.660,0.259,0.263,105.07,104.14,86.79,80.0,0.117
138,0022201200,1610612760,Oklahoma City,Thunder,OKC,thunder,1629652,Luguentz,Dort,L. Dort,...,18.2,0.143,0.285,0.200,0.202,109.96,107.09,89.24,42.0,0.000
139,0022201172,1610612760,Oklahoma City,Thunder,OKC,thunder,1629652,Luguentz,Dort,L. Dort,...,0.0,0.357,0.421,0.177,0.186,99.25,97.83,81.53,81.0,0.033


In [10]:
#conn = allintwo.connect_to_rds('thebigone', 'ajwin', 'CharlesBark!23', 'nba-rds-instance.c9wwc0ukkiu5.us-east-1.rds.amazonaws.com')
conn.close()

In [4]:
query = "SELECT * FROM mastergames;"  # Example query: fetch 10 rows from the 'games' table

allgames = allintwo.query_database_to_dataframe(conn, query)
allgameslist = allgames['gameid'].to_list()

tablegames = allintwo.fetch_table_data(conn,'boxscoreadvancedv3_0')
tablegameslist = tablegames['gameid'].to_list()
difference = set(allgameslist) - set(tablegameslist)
result = list(difference)

c:\Users\ajwin\Projects\NBA\thebigone\allintwo.py:103: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframe = pd.read_sql(query, conn)


In [54]:
result_df[result_df['gameid']=='0010500005']


,seasonid,teamid,teamabbreviation,teamname,gameid,gamedate,matchup,wl,min,pts,...,ftpct,oreb,dreb,reb,ast,stl,blk,tov,pf,plusminus
72092,12005,1610612742,DAL,Dallas Mavericks,0010500005,2005-10-11,DAL @ SAC,W,240,96,...,0.926,9.0,27.0,36,17,9.0,4,25,23,NaN
72097,12005,1610612758,SAC,Sacramento Kings,0010500005,2005-10-11,SAC vs. DAL,L,240,83,...,0.818,10.0,16.0,26,20,11.0,2,20,27,NaN


In [50]:
#nbaapi.boxscoreadvancedv2.BoxScoreAdvancedV2(game_id='0029600670').get_data_frames()[0]
nbaapi.BoxScoreAdvancedV2(game_id='0010500005').get_json()

'{"resource": "boxscore", "parameters": {"GameID": "0010500005", "StartPeriod": 0, "EndPeriod": 0, "StartRange": 0, "EndRange": 0, "RangeType": 0}, "resultSets": [{"name": "PlayerStats", "headers": [], "rowSet": []}, {"name": "TeamStats", "headers": [], "rowSet": []}]}'

In [55]:
#not working ids
nwi = ['0010500005']

hsbs = nbaapi.hustlestatsboxscore.HustleStatsBoxScore(game_id=nwi[0]).get_data_frames()[0]
hsbs

,GAME_ID,HUSTLE_STATUS
0,0010500005,0


In [56]:
570*60


34200